# MMA 662 - Assignment 2
## Instructions:
1. Answer the questions in this Jupyter Notebook file and upload it in the Assignment 2 entry under the Assignments tab in MyCourses. Answer each question in its own place. Late submissions are NOT accepted. Also, files not in the correct format (other than a Jupyter Notebook) are NOT accepted.
2. You can submit your answer file multiple times, but only the last submission is kept and graded.
3. For each question, make sure to print all the results that the question asks. Also, prevent printing unnecessary information.
4. We only grade the final answer, not the code, for coding questions. However, as the coding part of Question 7 can be a bit challenging, we will give **partial credit** for **concise and correct** explanations provided instead of the code only for this question. (Simple criterion: Your explanations should shed light on things that are not obvious from the question itself!) 
5. Simple explanations are not enough for formulation questions, and you have to write the exact mathematical formulation of the problem. If you are using an additional variable, introduce it clearly.

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

# Problem 1: The Apportionment Problem

## The Story:
The problem of allocating parliamentary seats in a *fair* way is a political matter in democratic societies. Suppose a country with a total population of $N = \sum_i p_i$ tries to allocate its $s$ available seats to its provinces, each with a population of $p_i$. The *common sense* says that the *fair* way to distribute these seats is to allocate each province a seat quota proportional to its population:
$$q_i = s \times \frac{p_i}{\sum_i p_i}.$$

Although this solution seems appealing, it ignores that the number of allocated seats must be **integers**. For instance, in a country with 30 million population and 130 parliament seats, the seat quota for a province with a population of 5 million is $q_i = \frac{5 \times 130}{30} = 21.\overline{66}$, which is impossible. In this problem, we will use two methods that you are familiar with to address this issue - plus some additional considerations.

## The Data:
The country we are working on has 20 provinces, and its parliament has 601 seats. In the attached file *assignment2_data.csv*, you can find the dataset for this problem. The first column is the province's name, the second column is the province's population, and the third column is the quota ($q_i$) for the province. The quota for each province is the continuous value we calculated using the formula above as $q_i = s \times \frac{p_i}{\sum_i p_i}$. Read the dataset using Python's Pandas package - or any package you choose - and extract the relevant information. (To ensure everyone is working with the same data, do not calculate the quotas again yourself; just use the given $q_i$ s.) For example, in this dataset, the population of province $P4$ is 3200000, and the quota for this province is 13.16359.

## Question 1: The Min Sum Approach: (7.5 points)

You have seen this approach in your first quiz. In this approach, the goal is to minimize the sum of absolute gaps between the number of allocated seats to each province ($a_i$) and its quota ($q_i$) while leaving no seats unfilled. That is, we seek to find allocations $a_0$ to $a_{19}$ to
$$\min_{a_0, \dots, a_{19}} \sum_{i=0}^{19} |q_i - a_i| \\ 
   s.t. \qquad \sum_{i=0}^{19} a_i = s \\
   a_i \in Z^+ \: for \: i = 0, \dots, 19.$$
(The last line means the number of allocated seats to each province has to be a non-negative integer.)

Linearize the above problem using the method you learned in the quiz, write its code using Gurobi, and solve it. Report the sum of absolute gaps for the optimal solution and the number of seats allocated to each province.

(Hint: notice that when you are linearizing the absolute value, the auxiliary variables you must introduce should be continuous, not discrete. This is because while the allocations are discrete, the gap between allocations and the quotas is continuous.)

###   <font color='blue'>with using approach we learned from the quiz</font> 


In [3]:
# Read data from the csv file:

data = pd.read_csv('assignment2_data.csv')
Province = data["Province"].values
Population = data["Population"].values
q = data["Quota"].values

Num = len(Province)

# Total seats:
s = 601

###############################################

# Create a new model
model = gb.Model("Question1")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Variables
a = model.addVars(Num, vtype=gb.GRB.INTEGER, name="a")  # Number of allocated seats for each province
z = model.addVars(Num, vtype=gb.GRB.CONTINUOUS, name="z")  # Absolute difference between allocated seats and quotas

# Objective: Minimize the sum of absolute gaps
model.setObjective(z.sum(), gb.GRB.MINIMIZE)

# Constraints
# 1. Total allocated seats must be equal to s
model.addConstr(a.sum() == s, "TotalSeats")

# 2. Linearized absolute value constraints
for i in range(Num):
    model.addConstr(z[i] >= q[i] - a[i], f"AbsDiff1_{i}")
    model.addConstr(z[i] >= a[i] - q[i], f"AbsDiff2_{i}")

# Solve the model
model.optimize()

# Print the results
if model.status == gb.GRB.OPTIMAL:
    allocated_seats = [int(a[i].x) for i in range(Num)]
    print("Sum of absolute gaps:", model.objVal)
    for i in range(Num):
        print(f"Province {Province[i]}: Allocated Seats = {allocated_seats[i]}")


Restricted license - for non-production use only - expires 2024-10-28
Set parameter MIPGap to value 1e-05
Sum of absolute gaps: 4.61599999995958
Province P0: Allocated Seats = 22
Province P1: Allocated Seats = 38
Province P2: Allocated Seats = 22
Province P3: Allocated Seats = 33
Province P4: Allocated Seats = 13
Province P5: Allocated Seats = 36
Province P6: Allocated Seats = 33
Province P7: Allocated Seats = 42
Province P8: Allocated Seats = 15
Province P9: Allocated Seats = 25
Province P10: Allocated Seats = 48
Province P11: Allocated Seats = 24
Province P12: Allocated Seats = 29
Province P13: Allocated Seats = 26
Province P14: Allocated Seats = 20
Province P15: Allocated Seats = 44
Province P16: Allocated Seats = 29
Province P17: Allocated Seats = 39
Province P18: Allocated Seats = 26
Province P19: Allocated Seats = 37


## Question 2: The Min Max Approach: (7.5 points)

You are familiar with this approach too! Here, instead of the sum of absolute gaps, we want to minimize the absolute value of the largest gap while leaving no seats unfilled. In other words, we want to find the allocations $a_0$ to $a_{19}$ to:
$$\min_{a_0, \dots, a_{19}} \max_{i} |q_i - a_i| \\ 
   s.t. \qquad \sum_{i=0}^{19} a_i = s \\
   a_i \in Z^+ \: for \: i = 0, \dots, 19.$$

Linearize the above problem using the method you learned in the quiz, write its code using Gurobi, and solve it. Report the maximum absolute gap in the optimal solution, the number of seats allocated to each province, and the sum of absolute gaps in the optimal solution.

In [4]:
# Read data from the csv file:

Province = data["Province"].values
Population = data["Population"].values
q = data["Quota"].values

Num = len(Province)

# Total seats:
s = 601

###############################################

# Create a new model
model = gb.Model("Question2")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

###############################################
# Your code here:

# Variables
a = model.addVars(Num, vtype=gb.GRB.INTEGER, name="a")  # Number of allocated seats for each province
M = model.addVar(vtype=gb.GRB.CONTINUOUS, name="M")  # Maximum absolute gap

# Objective: Minimize the maximum absolute gap
model.setObjective(M, gb.GRB.MINIMIZE)

# Constraints
# 1. Total allocated seats must be equal to s
model.addConstr(a.sum() == s, "TotalSeats")

# 2. Linearized constraints to capture the maximum absolute gap
for i in range(Num):
    model.addConstr(M >= q[i] - a[i], f"MaxAbsDiff1_{i}")
    model.addConstr(M >= a[i] - q[i], f"MaxAbsDiff2_{i}")

# Solve the model
model.optimize()

# Print the results
if model.status == gb.GRB.OPTIMAL:
    allocated_seats = [int(a[i].x) for i in range(Num)]
    print("Maximum absolute gap:", M.x)
    total_abs_gap = sum([abs(allocated_seats[i] - q[i]) for i in range(Num)])
    print("Sum of absolute gaps:", total_abs_gap)
    for i in range(Num):
        print(f"Province {Province[i]}: Allocated Seats = {allocated_seats[i]}")

Set parameter MIPGap to value 1e-05
Maximum absolute gap: 0.504449999999997
Sum of absolute gaps: 4.616000000000007
Province P0: Allocated Seats = 22
Province P1: Allocated Seats = 38
Province P2: Allocated Seats = 22
Province P3: Allocated Seats = 33
Province P4: Allocated Seats = 13
Province P5: Allocated Seats = 36
Province P6: Allocated Seats = 33
Province P7: Allocated Seats = 42
Province P8: Allocated Seats = 15
Province P9: Allocated Seats = 25
Province P10: Allocated Seats = 48
Province P11: Allocated Seats = 24
Province P12: Allocated Seats = 29
Province P13: Allocated Seats = 26
Province P14: Allocated Seats = 20
Province P15: Allocated Seats = 44
Province P16: Allocated Seats = 29
Province P17: Allocated Seats = 39
Province P18: Allocated Seats = 26
Province P19: Allocated Seats = 37


## Question 3: Political Considerations: (20 points)

Due to political reasons, some considerations should be addressed when allocating seats to each province. Here, we want to implement three of them:
1. The number of seats allocated to the two least populated provinces ($P4 \ \text{and} \ P8$) must be equal.
2. The number of seats allocated to each province with more than 10,000,000 population ($P7, P10, \ \text{and} \ P15$) should not exceed its quota.
3. If the number of seats allocated to $P0$ is more than its quota, then the number of seats allocated to $P2$ must be more than its quota as well.

Now answer the following questions:

### Question 3.1: Mathematical Formulation of the Considerations:

Write the mathematical formulation for each of the above considerations using the techniques you've learned in the course in the course in the following Markdown cell. (Your writing should be clear about which formula is for which consideration.) (10 points)

**Your formulations:**

### First Constraint:
- $a_4$ = $a_8$
### Second Constraint:
- $a_7$ <= $q_7$
- $a_{10}$ <= $q_{10}$
- $a_{15}$ <= $q_{15}$

### Third Constraint:
Let M be considered as a very large positive constant & y be a binary variable that takes fhte values of y = 1 if a0 > q0, y = 0 , otherwise; 

- $a_0$ <= $q_0$ + M*y, 
- $a_2$ >= $q_2$ - M*(1-y)

### Question 3.2: Implementing the Considerations:

Based on the first approach (minimizing the sum of absolute gaps while filling all the seats) and by adding these considerations as new constraints. Find the sum of absolute gaps for the optimal solution using Gurobi and report it. (10 points)

In [6]:
# Read data from the csv file:

Province = data["Province"].values
Population = data["Population"].values
q = data["Quota"].values

Num = len(Province)

# Total seats:
s = 601

###############################################

# Create a new model
model = gb.Model("Question3")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

###############################################
# Your code here:
# Variables
a = model.addVars(Num, vtype=gb.GRB.INTEGER, name="a")  # Number of allocated seats for each province
z = model.addVars(Num, vtype=gb.GRB.CONTINUOUS, name="z")  # Absolute difference between allocated seats and quotas

# Objective: Minimize the sum of absolute gaps
model.setObjective(z.sum(), gb.GRB.MINIMIZE)

# Constraints
# 1. Total allocated seats must be equal to s
model.addConstr(a.sum() == s, "TotalSeats")

# 2. Linearized absolute value constraints
for i in range(Num):
    model.addConstr(z[i] >= q[i] - a[i], f"AbsDiff1_{i}")
    model.addConstr(z[i] >= a[i] - q[i], f"AbsDiff2_{i}")

# 3. Number of seats allocated to P4 and P8 must be equal
model.addConstr(a[4] == a[8], "EqualSeats_P4_P8")

# 4. Number of seats for provinces with more than 10,000,000 population should not exceed its quota
for i in [7, 10, 15]:
    model.addConstr(a[i] <= q[i], f"QuotaLimit_{i}")

# 5. If a0 exceeds its quota, then a2 must also exceed its quota
M = 1e6  # A large constant
b = model.addVar(vtype=gb.GRB.BINARY, name="b")
model.addConstr(a[0] - q[0] <= M * b, "BinaryTrigger")
model.addConstr(a[2] - q[2] >= -M * (1 - b), "ConditionalConstraint")

# Solve the model
model.optimize()

# Print the results
if model.status == gb.GRB.OPTIMAL:
    allocated_seats = [int(a[i].x) for i in range(Num)]
    print("Sum of absolute gaps:", model.objVal)
    for i in range(Num):
        print(f"Province {Province[i]}: Allocated Seats = {allocated_seats[i]}")

Set parameter MIPGap to value 1e-05
Sum of absolute gaps: 6.574920000000006
Province P0: Allocated Seats = 22
Province P1: Allocated Seats = 38
Province P2: Allocated Seats = 23
Province P3: Allocated Seats = 33
Province P4: Allocated Seats = 14
Province P5: Allocated Seats = 36
Province P6: Allocated Seats = 33
Province P7: Allocated Seats = 41
Province P8: Allocated Seats = 14
Province P9: Allocated Seats = 25
Province P10: Allocated Seats = 48
Province P11: Allocated Seats = 24
Province P12: Allocated Seats = 29
Province P13: Allocated Seats = 26
Province P14: Allocated Seats = 20
Province P15: Allocated Seats = 44
Province P16: Allocated Seats = 29
Province P17: Allocated Seats = 39
Province P18: Allocated Seats = 26
Province P19: Allocated Seats = 37


# Problem 2: Designing the Floor Plan

## The Story and Data:

You are responsible for designing the floor plan of a shopping mall. The management team wants to include stores and facilities that maximize its expected annual profit while satisfying some constraints. In the following table, called *the original table*, you can see the yearly profit expected from one unit of each type of store/facility, the area it occupies, and the maximum number of units you can build from each type. (Notice that the mall can have more than one unit of each kind of store/facility as long as you do not pass the maximum number limit.)

| Facility/Store | Expected Annual Profit (\$) | Occupied Area ($m^2$) | Max Number of Units |
| :--: | :--: | :--: | :--: |
| Restaurant | 300,000 | 500 | 5 |
| Clothing Store | 800,000 | 1,700 | 15 |
| Tech Store | 1,000,000 | 1,000 | 5 |
| Pharmacy | 500,000 | 900 | 5 |
| Department Store | 2,200,000 | 4,300 | 3 |
| Toy Store | 500,000 | 1,400 | 5 |
| Toilet | -25,000 | 200 | 3 |
| Security Division | -100,000 | 300 | 3 |

There are two fundamental constraints that you need to consider when designing the floor plan **in all questions**:

1. The total area of all stores and facilities should be at most 20,000 $m^2$. (Just so you know, Montreal's Eaton Center's total retail floor area is 45,000 $m^2$.)
2. The mall should have at least one complete toilet and one complete security division. (By "at least one complete," we mean $\geq 1$.)

For example, we decided to have 2 restaurants, 2 clothing stores, 1 tech store, 3 pharmacies, 3 toy stores, 1 toilet, and 2 security divisions in the mall. The expected annual profit of this floor plan is:
$$2 \times \$300,000 + 2 \times \$800,000 + 1 \times \$1,000,000 + 3 \times \$500,000 + 3 \times \$500,000 - 1 \times \$25,000 - 2 \times \$100,000 = \$5,975,000$$

## Question 4: Using Continuous Variables: (15 points)

Suppose you can have continuous numbers of each type of store/facility. For instance, you are allowed to have 3.45 units of pharmacies in your mall. Remember to consider the two fundamental constraints.

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as a linear programming problem, solve it, and report the results using Gurobi or by hand. Remember, if you solve it by hand, you must provide solid, convincing explanations. 

In [7]:
Type = ['Restaurant', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [300000, 800000, 1000000, 500000, 2200000, 500000, -25000, -100000]

Area = [500, 1700, 1000, 900, 4300, 1400, 200, 300]

Max_No = [5, 15, 5, 5, 3, 5, 3, 3]

Num = len(Type)

###############################################

# Create a new model
model = gb.Model("Question4")
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

###############################################
# Your code here:

# Variables
x = model.addVars(Num, vtype=gb.GRB.CONTINUOUS, name="x")

# Objective: Maximize expected annual profit
model.setObjective(sum(Profit[i] * x[i] for i in range(Num)), gb.GRB.MAXIMIZE)

# Constraints
# 1. Total occupied area constraint
model.addConstr(sum(Area[i] * x[i] for i in range(Num)) <= 20000, "TotalArea")

# 2. Minimum number of toilets and security divisions
model.addConstr(x[6] >= 1, "MinToilets")
model.addConstr(x[7] >= 1, "MinSecurity")

# 3. Do not exceed the max number for each facility/store
for i in range(Num):
    model.addConstr(x[i] <= Max_No[i], f"Max_{Type[i]}")

# Solve the model
model.optimize()

# Print the results
if model.status == gb.GRB.OPTIMAL:
    print("Maximum expected annual profit: $", model.objVal)
    total_area = sum(Area[i] * x[i].x for i in range(Num))
    print("Total occupied area:", total_area, "m^2")
    for i in range(Num):
        print(f"{Type[i]}: {x[i].x} units")

Maximum expected annual profit: $ 12712209.30232558
Total occupied area: 20000.0 m^2
Restaurant: 5.0 units
Clothing Store: 0.0 units
Tech Store: 5.0 units
Pharmacy: 5.0 units
Dept Store: 1.744186046511628 units
Toy Store: 0.0 units
Toilet: 1.0 units
Security: 1.0 units


## Question 5: Using Integer Variables: (10 points)

In realistic settings, the number of stores/facilities should be discrete. Again, we are interested in the maximum expected annual profit and the optimal combination. Remember to consider the two fundamental constraints.

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as an integer programming problem, solve it, and report the results using Gurobi.

In [8]:
Type = ['Restaurant', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [300000, 800000, 1000000, 500000, 2200000, 500000, -25000, -100000]

Area = [500, 1700, 1000, 900, 4300, 1400, 200, 300]

Max_No = [5, 15, 5, 5, 3, 5, 3, 3]

Num = len(Type)

###############################################

# Create a new model
model = gb.Model("Question5")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

###############################################
# Your code here:

# Variables: Now making them INTEGER
x = model.addVars(Num, vtype=gb.GRB.INTEGER, name="x")

# Objective: Maximize expected annual profit
model.setObjective(sum(Profit[i] * x[i] for i in range(Num)), gb.GRB.MAXIMIZE)

# Constraints
# 1. Total occupied area constraint
model.addConstr(sum(Area[i] * x[i] for i in range(Num)) <= 20000, "TotalArea")

# 2. Minimum number of toilets and security divisions
model.addConstr(x[6] >= 1, "MinToilets")
model.addConstr(x[7] >= 1, "MinSecurity")

# 3. Do not exceed the max number for each facility/store
for i in range(Num):
    model.addConstr(x[i] <= Max_No[i], f"Max_{Type[i]}")

# Solve the model
model.optimize()

# Print the results
if model.status == gb.GRB.OPTIMAL:
    print("Maximum expected annual profit: $", model.objVal)
    total_area = sum(Area[i] * x[i].x for i in range(Num))
    print("Total occupied area:", total_area, "m^2")
    for i in range(Num):
        print(f"{Type[i]}: {x[i].x} units")

Set parameter MIPGap to value 1e-05
Maximum expected annual profit: $ 12475000.0
Total occupied area: 19700.0 m^2
Restaurant: 4.0 units
Clothing Store: -0.0 units
Tech Store: 5.0 units
Pharmacy: 4.0 units
Dept Store: 2.0 units
Toy Store: -0.0 units
Toilet: 1.0 units
Security: 1.0 units


## Question 6: Some Additional Constraints: (20 points)

We are still working based on the setup of Question 5, where we decided to use discrete decision variables. In addition to the two fundamental constraints, we want to maximize the expected profit while adding these four constraints to our model:
1. The number of toilets in the mall should be at least half the number of restaurants.
2. The mall should have at least one complete department store or one complete tech store, or both.
3. The total area the clothing stores occupy should not exceed 5,000 $m^2$.
4. If the mall has more than two complete toy stores, the number of security divisions should be at least two.

Answer the following questions:

### Question 6.1: Mathematical Formulation of the Additional Constraints:

Write the mathematical formulation for each of the above constraints using the techniques you've learned in the course in the following Markdown cell. (Your writing should be clear about which formula is for which one of the additional constraints.) (10 points)

**Your Formulations:**
### First Constraint:
- $2 \cdot X(\text{toilet}) - X(\text{restaurant}) \geq 0$

### Second Constraint:
- $X(\text{tech store}) + X(\text{department store}) \geq 1$

### Third Constraint:
- $ \text{Area}(\text{clothing store}) \cdot X(\text{clothing store}) \leq 5000$

### Fourth Constraint:
- $X(\text{toy store}) \leq 2 + M \cdot b$
- $X(\text{security}) \geq 2b$

### Question 6.2: Implementing the Additional Constraints:

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as a mixed integer programming problem (assume the number of each store/facility is an integer), solve it, and report the results using Gurobi. (10 points)

In [9]:
Type = ['Restaurant', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [300000, 800000, 1000000, 500000, 2200000, 500000, -25000, -100000]

Area = [500, 1700, 1000, 900, 4300, 1400, 200, 300]

Max_No = [5, 15, 5, 5, 3, 5, 3, 3]

Num = len(Type)

###############################################

# Create a new model
model = gb.Model("Question6")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

###############################################
# Your code here:

# Variables
x = model.addVars(Num, vtype=gb.GRB.INTEGER, name="x")
y = model.addVar(vtype=gb.GRB.BINARY, name="y")  # Additional binary variable for the 4th constraint

# Objective: Maximize expected annual profit
model.setObjective(sum(Profit[i] * x[i] for i in range(Num)), gb.GRB.MAXIMIZE)

# Constraints
# 1. Total occupied area constraint
model.addConstr(sum(Area[i] * x[i] for i in range(Num)) <= 20000, "TotalArea")

# 2. Minimum number of toilets and security divisions
model.addConstr(x[6] >= 1, "MinToilets")
model.addConstr(x[7] >= 1, "MinSecurity")

# 3. Do not exceed the max number for each facility/store
for i in range(Num):
    model.addConstr(x[i] <= Max_No[i], f"Max_{Type[i]}")

# Additional Constraints
# 1. Toilets at least half the number of restaurants
model.addConstr(2*x[6]- x[0] >= 0, "Toilets_Half_Restaurants")

# 2. At least one department store or one tech store, or both
model.addConstr(x[4] + x[2] >= 1, "Dept_or_Tech_Store")

# 3. Clothing store area should not exceed 5000 m^2
model.addConstr(Area[1] * x[1] <= 5000, "Clothing_Store_Area")

# 4. If more than two toy stores, at least two security divisions
M = 1e6  # A large constant
model.addConstr(x[5] - 2 <= M * y, "BinaryTrigger")
model.addConstr(x[7] >= 2 * y, "Conditional_Security")

# Solve the model
model.optimize()

# Print the results
if model.status == gb.GRB.OPTIMAL:
    print("Maximum expected annual profit: $", model.objVal)
    total_area = sum(Area[i] * x[i].x for i in range(Num))
    print("Total occupied area:", total_area, "m^2")
    for i in range(Num):
        print(f"{Type[i]}: {x[i].x} units")

Set parameter MIPGap to value 1e-05
Maximum expected annual profit: $ 12450000.0
Total occupied area: 19900.0 m^2
Restaurant: 4.0 units
Clothing Store: -0.0 units
Tech Store: 5.0 units
Pharmacy: 4.0 units
Dept Store: 2.0 units
Toy Store: -0.0 units
Toilet: 2.0 units
Security: 1.0 units


## Question 7: The Case for Diminishing Returns: (15 points)

**Diminishing return to scale**: We expect the marginal profit of each additional unit from each store to decrease. That is, while a single pharmacy in the mall brings about \$500,000 annual profit, having two pharmacies in the mall creates \$800,000, and three pharmacies make \$900,000 together. Therefore, the marginal profit of adding a new pharmacy is \$500,000 when there is no pharmacy, \$300,000 when there is one pharmacy, and \$100,000 when there are already two pharmacies in the mall. 

Considering this phenomenon, our expectation for the annual profit created by the different number of restaurants, technology stores, and pharmacies is updated as in the following table: (Do not forget that the numbers in the following table represent the profit we expect all the stores to create together, not the profit produced by each of them. So, for example, we expect three restaurants to make \$750,000 altogether.)

| Store | Profit of one unit ($) | Two units | Three units | Four units | Five units |
| :--: | :--: | :--: | :--: | :--: | :--: |
| Restaurant | 300,000 | 550,000 | 750,000 | 850,00 | 900,000 |
| Tech Store | 1,000,000 | 1,750,000 | 2,250,000 | 2,500,000 | 2,600,000 |
| Pharmacy | 500,000 | 800,000 | 900,000 | 950,000 | 950,000 |

The maximum number limit for each type of store/facility remains the same as the limits depicted in the original table, and the expected profit for stores/facilities other than restaurants, technology stores, and pharmacies is estimated linearly similar to the previous questions.

For example, suppose we decided to have 2 restaurants, 2 clothing stores, 1 tech store, 3 pharmacies, 3 toy stores, 1 toilet, and 2 security divisions in the mall. Then, the expected profit of this decision is:
$$\$550,000 + 2 \times \$800,000 + \$1,000,000 + \$900,000 + 3 \times \$500,000 - 1 \times \$25,000 - 2 \times \$100,000 = \$5,325,000$$

In addition to diminishing return to scale and the two fundamental constraints, we still want to entertain the constraints we introduced in Question 6.

What is the maximum expected annual profit, and what combination of stores/facilities brings about this profit? What is the total occupied area? Model it as a mixed integer programming problem (assume the number of each store/facility is an integer), solve it, and report the results using Gurobi.

In [11]:
Type = ['Restaurant', 'Clothing Store', 'Tech Store', 'Pharmacy', 'Dept Store', 'Toy Store', 'Toilet', 'Security']

Profit = [300000, 800000, 1000000, 500000, 2200000, 500000, -25000, -100000]

Area = [500, 1700, 1000, 900, 4300, 1400, 200, 300]

Max_No = [5, 15, 5, 5, 3, 5, 3, 3]

Num = len(Type)

Rest_profit = [0, 300000, 550000, 750000, 850000, 900000]
Tech_profit = [0, 1000000, 1750000, 2250000, 2500000, 2600000]
Pharma_profit = [0, 500000, 800000, 900000, 950000, 950000]

###############################################

# Create a new model
model = gb.Model("Question7")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

###############################################
# Your code here:

## Decision variables
x = model.addVars(Num, vtype = GRB.INTEGER, name = [f'Number of facilities of {i} to build' for i in Type])
y = model.addVar(vtype = GRB.INTEGER, name = 'y')
R = model.addVars(len(Rest_profit), vtype = GRB.BINARY, name = [f'{i} restaurants build' for i in range(len(Rest_profit))])
T = model.addVars(len(Tech_profit), vtype = GRB.BINARY, name = [f'{i} tech build' for i in range(len(Rest_profit))])
P = model.addVars(len(Pharma_profit), vtype = GRB.BINARY, name = [f'{i} pharma build' for i in range(len(Rest_profit))])
## Objective function

marginal_profits = sum(R[i]*Rest_profit[i] + T[i] * Tech_profit[i] + P[i] * Pharma_profit[i] for i in range(len(Rest_profit)))
model.setObjective(sum(Profit[i] * x[i] for i in [1,4,5,6,7]) + marginal_profits , GRB.MAXIMIZE)

## Constraints 


### Max number of units allowed

model.addConstrs(x[i]<= Max_No[i] for i in range(Num))

### Total area

model.addConstr(sum(Area[i] * x[i] for i in range(Num))<= 20000)

## At least one toilet

model.addConstr(x[6]>= 1)

## At lears one complete security division

model.addConstr(x[7]>= 1)

## The number of toilets in the mall should be at least half the number of restaurants.

model.addConstr(x[6]>= 0.5*x[0])

## The mall should have at least one complete department store or one complete tech store, or both.

model.addConstr(x[4] + x[2]>=1)

## The total area the clothing stores occupy should not exceed 5,000 $m^2$.

model.addConstr(x[1]*Area[1]<= 5000)

#If the mall has more than two complete toy stores, the number of security divisions should be at least two.

M = 10000000
model.addConstr(x[5] - 2 <= M * y)
model.addConstr(x[7] >= 2 * y)

# Constraints for marginal profits

model.addConstr(sum(R[i] for i in range(len(Rest_profit))) == 1 )
model.addConstr(sum(T[i] for i in range(len(Tech_profit))) == 1 )
model.addConstr(sum(P[i] for i in range(len(Pharma_profit))) == 1 )
model.addConstr(sum(i*R[i] for i in range(len(Rest_profit))) == x[0])
model.addConstr(sum(i*T[i] for i in range(len(Tech_profit))) == x[2])
model.addConstr(sum(i*P[i] for i in range(len(Pharma_profit))) == x[3])

# Optimize

model.optimize()

## Results

print('Maximum profit:',model.ObjVal)
print('Total occupied area:',np.sum(np.array([i.x for i in model.getVars() if 'facilities' in i.VarName]) * np.array(Area)))
for var in model.getVars():
    if var.x > 0:
        print(var.VarName,':',var.x)


Set parameter MIPGap to value 1e-05
Maximum profit: 10575000.0
Total occupied area: 20000.0
Number of facilities of Restaurant to build : 2.0
Number of facilities of Clothing Store to build : 1.0
Number of facilities of Tech Store to build : 3.0
Number of facilities of Pharmacy to build : 1.0
Number of facilities of Dept Store to build : 3.0
Number of facilities of Toilet to build : 1.0
Number of facilities of Security to build : 1.0
2 restaurants build : 1.0
3 tech build : 1.0
1 pharma build : 1.0


# Problem 3: BlancedMilk Needs Your Help (Again!)

We have been through the problems BalancedMilk faces after it lost its largest supplier multiple times. (You may review Assignment and Quiz 1 if you forgot them.) The issue is not resolved yet, and $S_8$ is still shut down. BalancedMilk managers are so excited that now you know how to implement logical constraints using MIP, because they want you to help them with their new problem! They gave you the following information:
1. BalancedMilk is **not** committed to distributing the milk supplied by the remaining supply centers, $S_1$ to $S_7$. (It has the option not to distribute the milk it receives from the remaining supply centers.) Additionally, it **must not** oversupply any demand market.
2. The cost of not supplying (not delivering) milk to each demand market is 15$/tonne.
3. $S_3$, $S_4$, and $S_5$ belong to the same company. The company has imposed the following constraint: BalancedMilk should utilize $\geq50\%$ (equal or more than $50\%$) of the capacity of at least two of the three supply centers owned by this company.
4. Because $D_5$ and $D_6$ are close, the firm has some flexibility in answering their demands. So either $D_5$ receives $\geq80\%$ of its demand, or $D_6$ receives $\geq60\%$ of its demand, or both.
5. A competitor is trying to damage BalancedMilk's reputation by focusing on the discrimination it imposes between different demand centers. This competitor is trying to exploit the difference between $D_2$ and $D_8$ and $D_9$. Therefore, to respond to this criticism, if $D_2$ receives $> 50\%$ of its demand, then $D_8$ and $D_9$ should both receive $\geq50\%$ of their demand as well.

(Trust me. I am not a recruiter for any dairy transportation company, and neither am I preparing the MMA cohort for an internship at one. It is just that BalancedMilk has so many unresolved issues!)

## Question 8: The formulation (15 points)
Write the mathematical formulation of this problem using the techniques you've learned in the course in the following Markdown cell. Use the variables we have already defined.

$x_{ij}$ shows the amount of milk (in tonnes) transported from supply center $i$ to demand center $j$. \
$c_{ij}$ is the transportation cost from supply center $i$ to demand center $j$. \
$S_i$ is the capacity of supply center $i$. \
$D_j$ is the demand of demand center $j$. 

We introduce the following auxiliary variables: \
$y_i$ shows the amount of milk (in tonnes) supplied by supply center $i$: $y_i = \sum_{j = 1}^{10} x_{ij} \; for \; i = 1, \dots, 7.$ \
$z_j$ shows the amount of milk (in tonnes) delivered to demand center $j$. $z_j = \sum_{i = 1}^7 x_{ij} \; for \; j = 1, \dots, 10$.

**The objective function is**: 

$\min \quad \quad [(\sum_{i=1}^{7} \sum_{j=1}^{10} c_{ij}*x{ij}) + 15*\sum_{j=1}^{10}(D_j-z_{j})]$
 

**The constraints are**: \
**Supply and demand constraints:** \


- $y_i = \sum_{j = 1}^{10} x_{ij} \; for \; i = 1, \dots, 7$ <br>
- $z_j = \sum_{i = 1}^7 x_{ij} \; for \; j = 1, \dots, 10$ <br>

- $y_i \leq S_i  \; for \; i = 1, \dots, 7$ <br>
- $z_j = \leq D_j \; for \; j = 1, \dots, 10$ <br>


#### Logical constraints: \
**First logical constraint:**
\begin{align*}
&y_3 \geq 0.5S_3b_1 \\
&y_4 \geq 0.5S_4b_2 \\
&y_5 \geq 0.5S_5b_3 \\
&b_1 + b_2 + b_3 \geq 2 \\
&b_1, b_2, b_3 \in \{0, 1\}
\end{align*}

**Second logical constraint:**
\begin{align*}
&z_5 \geq 0.8D_5 - Mb_4 \\
&z_6 \geq 0.6D_6 - M(1-b_4) \\
&b_4 \in \{0, 1\}
\end{align*}

**Third logical constraint:**
\begin{align*}
&z_2 \leq 0.5D_2 + Mb_5 \\
&z_8 \geq 0.5D_8 - M(1-b_5) \\
&z_9 \geq 0.5D_9 - M(1-b_5) \\
&b_5 \in \{0, 1\}
\end{align*}


## Question 9: The implementation (10 points)

Write the code for this problem using Gurobi and see if an optimal solution with these constraints exists. If yes, what are the lowest cost, the utilized capacity of each supply center, and the satisfied demand of each demand center?

(Remember that while the indices of suppliers and demand markets start from 1, the indices of the variables in Python begin from 0.)

In [15]:
Cost = [[20, 49, 16, 30,  8, 35, 21, 40, 10, 12],
        [15, 53, 7, 20, 47, 11, 16, 17, 15, 44],
        [22, 25, 42, 22, 31, 9, 11, 29, 20, 5],
        [45,  6, 33, 35, 49, 25, 30, 47, 32, 31],
        [9, 12, 41, 15, 38, 14, 53, 22, 12, 13],
        [21, 24, 32, 49, 5, 47, 30, 23, 37, 8],
        [12, 19,  5, 28, 47, 39, 15, 35, 9, 51]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7']
Supply = [110, 80, 100, 240, 100, 280, 130]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [90, 100, 150, 190, 180, 240, 210, 90, 160, 70]


n_supply = len(SupplyCenter)
n_demand = len(DemandMarket)
Range_supply = range(n_supply)
Range_demand = range(n_demand)

##############################

# Create a new model
model = gb.Model("Question9")
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0
model.setParam('MIPGap', 0.00001)

###############################################
# Your code here:

#Variable x_ij
X = model.addVars(n_supply,n_demand,lb=0,vtype=GRB.CONTINUOUS,name=[f'x_{i}_{j}' for i in range(1,n_supply+1) for j in range(1,n_demand+1)])

#Auxiliary variable Y
Y = model.addVars(n_supply,lb=0,vtype=GRB.CONTINUOUS,name=[f'Amount Supplied by S_{i}'for i in range(1,n_supply+1)])

#Auxiliary variable Z
Z = model.addVars(n_demand,lb=0,vtype=GRB.CONTINUOUS,name=[f'Amount Delivered to D_{j}'for j in range(1,n_demand+1)])

#Dummy variable b:
B = model.addVars(5,vtype=GRB.BINARY,name=[f'b_{k}' for k in range(1,6)])

#Objective function:

ship_cost = sum(Cost[i][j]*X[i,j] for i in Range_supply for j in Range_demand)
diff_in_dd = sum(Demand[j]-Z[j] for j in Range_demand)

exp = ship_cost + 15 *(diff_in_dd)

model.setObjective(exp,GRB.MINIMIZE)

# constraint for auxiliary:

model.addConstrs(Y[i] == sum(X[i,j] for j in Range_demand) for i in Range_supply)
model.addConstrs(Z[j] == sum(X[i,j] for i in Range_supply) for j in Range_demand)

#Supply and Demand constraint:

model.addConstrs(Y[i] <= Supply[i] for i in Range_supply)
model.addConstrs(Z[j] <= Demand[j] for j in Range_demand)

#First logical constraint:
model.addConstr(Y[2] >= 0.5*Supply[2]*B[0]) #y_3,s_3,b_1
model.addConstr(Y[3] >= 0.5*Supply[3]*B[1]) #y_4,s_4,b_2
model.addConstr(Y[3] >= 0.5*Supply[3]*B[1]) #y_5,s_5,b_3
model.addConstr(B[0] + B[1] + B[2] >= 2)

#Second logical constraint:
M = 1000000
model.addConstr(Z[4] >= 0.8*Demand[4] - M*B[3]) #z_5,d_5,b_4
model.addConstr(Z[5] >= 0.6*Demand[6] - M*(1-B[3])) #z_6,d_6,b_4

#Third logical constraint:
model.addConstr(Z[1] <= 0.5*Demand[1] + M*B[4]) #z_2,d_2,b_5
model.addConstr(Z[7] >= 0.5*Demand[7] - M*(1-B[4])) #z_8,d_8,b_5
model.addConstr(Z[8] >= 0.5*Demand[8] - M*(1-B[4])) #z_9,d_9,b_5

model.optimize()

if model.status == GRB.OPTIMAL:
    print('Minimum cost is: $',model.objVal)
    
    for v in model.getVars():
        if v.x > 0:
            print(v.varName,':',v.x)

else:
    print('Optimization was stopped with status %d' % model.status)


Minimum cost is: $ 15860.0
x_1_9 : 110.0
x_2_3 : 20.0
x_2_6 : 15.0
x_2_8 : 45.0
x_3_6 : 100.0
x_4_2 : 100.0
x_5_1 : 90.0
x_5_9 : 10.0
x_6_5 : 180.0
x_6_10 : 70.0
x_7_3 : 130.0
Amount Supplied by S_1 : 110.0
Amount Supplied by S_2 : 80.0
Amount Supplied by S_3 : 100.0
Amount Supplied by S_4 : 100.0
Amount Supplied by S_5 : 100.0
Amount Supplied by S_6 : 250.0
Amount Supplied by S_7 : 130.0
Amount Delivered to D_1 : 90.0
Amount Delivered to D_2 : 100.0
Amount Delivered to D_3 : 150.0
Amount Delivered to D_5 : 180.0
Amount Delivered to D_6 : 115.0
Amount Delivered to D_8 : 45.0
Amount Delivered to D_9 : 120.0
Amount Delivered to D_10 : 70.0
b_1 : 1.0
b_3 : 1.0
b_5 : 1.0
